In [15]:
# !pip install -r ./requirements.txt

In [50]:
# !pip uninstall virny -y

In [52]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [53]:
# !pip install lightgbm==3.3.5

In [54]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [56]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [57]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSEmploymentDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.preprocessing import get_simple_preprocessor, create_extra_test_sets
from source.experiment_interface import run_exp_iter_with_preprocessing_intervention, run_exp_iter_with_mult_set_and_preprocessing_intervention

## Define Input Variables

In [58]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'one_repair_lvl_many_models'
DB_COLLECTION_NAME = f'exp_{EXPERIMENT_NAME}'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.6]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'folk_ny_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [59]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [60]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [61]:
import uuid

custom_table_fields_dct = {
    'session_uuid': str(uuid.uuid4()),
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5


## Initialize custom objects

In [62]:
data_loader = ACSEmploymentDataset(state=['NY'], year=2018, with_nulls=False,
                                   subsample_size=20_000, subsample_seed=42)
data_loader.X_data.head()

,MAR,MIL,ESP,MIG,DREM,NATIVITY,DIS,DEAR,DEYE,SEX,RAC1P,RELP,CIT,ANC,SCHL,AGEP
0,5,4,0,1,2,1,2,2,2,1,1,2,1,1,15,42
1,1,4,0,1,2,1,2,2,2,2,1,0,1,2,21,59
2,2,4,0,1,2,2,2,2,2,2,1,0,4,4,19,78
3,5,4,0,1,2,1,2,2,2,1,1,10,1,4,14,19
4,1,2,0,1,2,1,2,2,2,1,1,0,1,1,17,87


In [63]:
data_loader.X_data.shape

(20000, 16)

In [ ]:
data_loader2 = ACSEmploymentDataset(state=['NY'], year=2017, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader2.X_data.head()

In [ ]:
data_loader2.X_data.shape

In [ ]:
data_loader3 = ACSEmploymentDataset(state=['CA'], year=2018, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader3.X_data.head()

In [ ]:
data_loader3.X_data.shape

In [ ]:
data_loader4 = ACSEmploymentDataset(state=['CA'], year=2017, with_nulls=False,
                                    subsample_size=20_000, subsample_seed=42)
data_loader4.X_data.head()

Create extra out of domain test sets

In [ ]:
column_transformer = get_simple_preprocessor(data_loader)
extra_data_loaders = [data_loader2, data_loader3, data_loader4]
extra_test_sets = create_extra_test_sets(extra_data_loaders, column_transformer, TEST_SET_FRACTION, seed=42)

## Run experiment iterations

### Experiment iteration 1

In [33]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
exp_iter_extra_test_sets = copy.deepcopy(extra_test_sets)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)
preprocessor = get_simple_preprocessor(exp_iter_data_loader)

In [34]:
run_exp_iter_with_mult_set_and_preprocessing_intervention(
    data_loader=exp_iter_data_loader,
    experiment_seed=experiment_seed,
    test_set_fraction=TEST_SET_FRACTION,
    db_writer_func=db_writer_func,
    fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
    extra_test_sets=exp_iter_extra_test_sets,
    column_transformer=preprocessor,
    models_params_for_tuning=models_params_for_tuning,
    metrics_computation_config=metrics_computation_config,
    custom_table_fields_dct=custom_table_fields_dct,
    with_tuning=True,
    # with_tuning=False,
    # tuned_params_df_paths=tuned_params_df_paths,
    save_results_dir_path=SAVE_RESULTS_DIR_PATH,
    verbose=True
)

2023-07-06 13:27:22 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[1.0]',
 'model_init_seed': 100,
 'session_uuid': 'e6bca5bf-8de7-46ce-bb59-4c2b28dcaaf5'}




2023-07-06 13:27:23 experiment_interface.py INFO    : The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 2917,  2234, 14396,  1781, 11102,   732,  7692, 10589, 16098,
             4920,  6601,  7611,  3825, 18755,  6862,  3847,  7256, 13711,
            12389,  9772],
           dtype='int64')


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

...

2023-07-06 13:55:11 experiment_interface.py INFO    : Models are tuned and saved to a file


2023/07/06, 13:55:11: Tuning for RandomForestClassifier is finished [F1 score = 0.7770552282956381, Accuracy = 0.7868333333333334]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

### Experiment iteration 3

### Experiment iteration 4

### Experiment iteration 5

### Experiment iteration 6

### Experiment iteration 7

### Experiment iteration 8

### Experiment iteration 9

### Experiment iteration 10